In [31]:
def parse_section(data_lines):
    data_dict = {}
    for line in data_lines:
        if line.strip() and not line.strip().startswith("usgs_subfault"):
            parts = line.split()
            key = (float(parts[0]), float(parts[1]))
            value = [float(parts[3]) + float(parts[4]), float(parts[5])]
            data_dict[key] = value
    return data_dict

# Initialize an empty dictionary to store dictionaries for each section
all_data_dicts = {}
current_section = None

geoclaw_data = r"C:\Users\ashle\Documents\GitHub\tsunamibayes\tsunamibayes\grad_approx_two_per.txt"

# Assuming the data is stored in a file 'wave_tol_test.ssv', you can read it line by line
with open(geoclaw_data, 'r') as file:
    section_lines = []  # To collect lines for each section
    for line in file:
        if line.strip().startswith("usgs_subfault"):
            # If marker line is found, determine the current section
            if current_section is not None:
                # Process the collected section_lines and update current_section
                section_dict = parse_section(section_lines)
                current_section.update(section_dict)
                # Store current_section in all_data_dicts
                all_data_dicts[parameter_value] = current_section

            # Initialize a new current_section for the next section
            parameter_value = line.strip().split('.', 1)[1].strip()
            current_section = {}
            section_lines = []  # Reset section_lines for the new section
        else:
            # Collect lines for the current section
            section_lines.append(line)

    # Process the last section if there are remaining lines
    if current_section is not None:
        section_dict = parse_section(section_lines)
        current_section.update(section_dict)
        all_data_dicts[parameter_value] = current_section

# Function to print the dictionary
def print_dictionary(data_dict):
    for key, value in data_dict.items():
        print(f"parameter value: {key}")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")

print_dictionary(all_data_dicts)

parameter value: latitude = -5.915551496
  (129.775, -4.5175): [2.5777540000000005, 1193.7653]
  (128.178, -3.691): [6.200237999999999, 2625.7169]
  (129.8965, -4.5248): [3.8798239999999993, 1312.0522]
  (127.113, -3.3815): [3.4210967000000005, 3483.046]
  (128.557, -3.543): [3.48315, 2680.2774]
  (128.657, -3.576): [8.0835106, 2441.8444]
  (128.562, -3.501): [2.1362450000000006, 3250.3275]
  (128.807, -3.6455): [3.5941570000000027, 2071.473]
  (128.921, -3.338): [5.2923288, 2478.6879]
parameter value: latitude = -5.914368504
  (129.775, -4.5175): [2.577568000000001, 1193.7655]
  (128.178, -3.691): [6.201094999999995, 2625.7165]
  (129.8965, -4.5248): [3.8818669999999997, 1312.0525]
  (127.113, -3.3815): [3.420684, 3483.0604]
  (128.557, -3.543): [3.4834510000000005, 2680.2765]
  (128.657, -3.576): [8.0823003, 2441.8445]
  (128.562, -3.501): [2.134798, 3250.351]
  (128.807, -3.6455): [3.595134999999999, 2071.4731]
  (128.921, -3.338): [5.293187, 2478.688]
parameter value: longitude = 1

In [32]:
def compare_inner_dicts(dict1, dict2, two_h):
    finite_diff = {}
    for key in dict1.keys():
        if key in dict2:
            finite_diff[key] = (float(dict1[key][-1]) - float(dict2[key][-1]))/ float(two_h)
    return finite_diff

def compare_outer_dicts(outer_dict):
    comparison_results = {}
    processed_keys = set()
    
    for key1 in outer_dict:
        prefix1, val1 = key1.split('=')[:2]
        prefix1 = prefix1.strip()
        val1 = float(val1.strip())
        
        if prefix1 not in processed_keys:
            processed_keys.add(prefix1)
            comparison_results[prefix1] = {}
            for key2 in outer_dict:
                prefix2, val2 = key2.split('=')[:2]
                prefix2 = prefix2.strip()
                val2 = float(val2.strip())
                
                if prefix1 == prefix2 and key1 != key2:
                    two_h = val1 - val2
                    differences = compare_inner_dicts(outer_dict[key1], outer_dict[key2], two_h)
                    if differences:
                        comparison_results[prefix1] = differences
    return comparison_results


# Compare outer dictionaries and print results
comparison_results = compare_outer_dicts(all_data_dicts)


switched_data = {}
for outer_key, inner_dict in comparison_results.items():
    for inner_key, value in inner_dict.items():
        if inner_key not in switched_data:
            switched_data[inner_key] = {}
        switched_data[inner_key][outer_key] = value
        
for key, section_dict in switched_data.items():
    print(f"Longitude: {key[0]}, Latitude: {key[1]}")
    for key2, values in section_dict.items():
        print(f"{key2}:{values}")
    print()       

Longitude: 129.775, Latitude: -4.5175
latitude:0.16906285076292252
longitude:294.2182558988093
length:0.0
width:0.0
slip:-0.0075529528061218905
strike:4.72992446087369
dip:-4.334837736207457
depth:0.0
rake:0.0

Longitude: 128.178, Latitude: -3.691
latitude:-0.33812570152584503
longitude:387.7820177572322
length:0.0
width:4.1371303365902075e-05
slip:0.049094193248379
strike:1.2339039462804307
dip:-2.1675307695370827
depth:0.0
rake:-0.05555555566783885

Longitude: 129.8965, Latitude: -4.5248
latitude:0.2535942761443838
longitude:417.0100036487518
length:0.0
width:0.0
slip:-0.0037764764030609453
strike:-0.0017079943888862357
dip:0.005818874549159095
depth:0.0
rake:0.0

Longitude: 127.113, Latitude: -3.3815
latitude:12.172525258005656
longitude:439.78806859644783
length:0.0005084099263830729
width:0.0009101686721684966
slip:2.4924744266556407
strike:-0.3545308352672627
dip:-1.6230183934621258
depth:0.001428570822291381
rake:0.11111111108304028

Longitude: 128.557, Latitude: -3.543
latitude

In [33]:
def compare_inner_dicts(dict1, dict2, two_h):
    finite_diff = {}
    for key in dict1.keys():
        if key in dict2:
            finite_diff[key] = (float(dict1[key][0]) - float(dict2[key][0]))/ float(two_h)
    return finite_diff

def compare_outer_dicts(outer_dict):
    comparison_results = {}
    processed_keys = set()
    
    for key1 in outer_dict:
        prefix1, val1 = key1.split('=')[:2]
        prefix1 = prefix1.strip()
        val1 = float(val1.strip())
        
        if prefix1 not in processed_keys:
            processed_keys.add(prefix1)
            comparison_results[prefix1] = {}
            for key2 in outer_dict:
                prefix2, val2 = key2.split('=')[:2]
                prefix2 = prefix2.strip()
                val2 = float(val2.strip())
                
                if prefix1 == prefix2 and key1 != key2:
                    two_h = val1 - val2
                    differences = compare_inner_dicts(outer_dict[key1], outer_dict[key2], two_h)
                    if differences:
                        comparison_results[prefix1] = differences
    return comparison_results

# Compare outer dictionaries and print results
comparison_results = compare_outer_dicts(all_data_dicts)   
    
switched_data = {}
for outer_key, inner_dict in comparison_results.items():
    for inner_key, value in inner_dict.items():
        if inner_key not in switched_data:
            switched_data[inner_key] = {}
        switched_data[inner_key][outer_key] = value


for key, section_dict in switched_data.items():
    print(f"Longitude: {key[0]}, Latitude: {key[1]}")
    for key2, values in section_dict.items():
        print(f"{key2}:{values}")
    print()

Longitude: 129.775, Latitude: -4.5175
latitude:-0.1572284512484389
longitude:0.4080135611773461
length:7.626148891953001e-06
width:-1.3652530083203559e-05
slip:0.1920715899078404
strike:0.02429012020015311
dip:-0.013504265011236796
depth:-1.4285708225324577e-05
rake:-0.010555555555533625

Longitude: 128.178, Latitude: -3.691
latitude:0.7244343156978711
longitude:-1.9366486256384388
length:2.796254595973894e-05
width:-8.68797368790836e-06
slip:0.46620601207502865
strike:-0.014905911029896437
dip:-0.0350497670435986
depth:-2.321427586020482e-05
rake:0.006666666665614011

Longitude: 129.8965, Latitude: -4.5248
latitude:1.72697702097715
longitude:0.6770326562880232
length:1.0168198531635176e-05
width:-4.05438772156283e-05
slip:0.29229927367022623
strike:0.06432306868070595
dip:-0.012313633757349953
depth:-2.8571416456993285e-05
rake:-0.0172222222221345

Longitude: 127.113, Latitude: -3.3815
latitude:-0.34886119263727416
longitude:-0.7825916747750093
length:1.5760707720873613e-05
width:7.03